In [ ]:
# Final Routing Interpolation script
# Built off of 2.4 NJDOT Truck Counts, but moved here to make for a cleaner file
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy
import folium
import geopyx
import geopandas as gpd
from geopy.geocoders import Nominatim

#### Data Import and Prep ----------------------------------------------------

In [ ]:
# Set working directory
# setwd("\\\\mspe-gis-file/GISProj/SJ/10306158 South Jersey Freight Profile/7.2_WIP")

In [ ]:
# Load Freight Trip Generation (Block Level Data)
SJTPO_blocksFTG = gpd.read_file("SJTPO_blocksFTG.RData")

# Load Pop and Other Base Data
baseData = pd.read_pickle("baseData.RData")

# Load original classification count location file
countDataJoin = pd.read_pickle("countDataJoin.Rdata")
countDataJoin = countDataJoin[['si_station_num', 'adt', 'truckADT', 'comboADT']]
countDataJoin['type'] = "Original"

# Import the new points added through ArcGIS
countDataJoinExtra = gpd.read_file("ArcGIS/CountLocEdits/CountLocsForEdit2.shp")
countDataJoinExtra = countDataJoinExtra.iloc[288:].copy()
countDataJoinExtra['si_station_num'] = countDataJoinExtra.index.map(lambda x: f"extra{x}")
countDataJoinExtra['type'] = countDataJoinExtra.apply(lambda row: "Collection" if row.name < 22 else row['siteName'], axis=1)

# Import the Cumberland County Truck Study Counts  
CCC_East = gpd.read_file("FromTeams/ClassificationCounts/ForGIS/CCC_East.shp")
CCC_West = gpd.read_file("FromTeams/ClassificationCounts/ForGIS/CCC_West.shp")

# Combine CCC East and West
CCC = pd.concat([CCC_East, CCC_West], ignore_index=True)
CCC['si_station_num'] = CCC['site2']
CCC['type'] = "CCC"

# Import FAF Points
fafNetworkSJTPOdots = gpd.read_file("fafNetworkSJTPOdots.RData")
fafNetworkSJTPOdots['si_station_num'] = fafNetworkSJTPOdots.index.map(lambda x: f"FAF_{x}")
fafNetworkSJTPOdots['adt'] = fafNetworkSJTPOdots['tot_trips_22_all'] / 0.07
fafNetworkSJTPOdots['truckADT'] = fafNetworkSJTPOdots['tot_trips_22_all']
fafNetworkSJTPOdots['comboADT'] = fafNetworkSJTPOdots['tot_trips_22_cu']
fafNetworkSJTPOdots['type'] = "FAF"

# Chart of ADT vs. Truck Percentage
plt.scatter(countDataJoin['adt'], countDataJoin['truckADT'] / countDataJoin['adt'])
plt.xlabel('ADT')
plt.ylabel('Truck Percentage')
plt.show()

# Combine all data together for analysis
countDataFinal = pd.concat([countDataJoin, countDataJoinExtra, CCC, fafNetworkSJTPOdots])
countDataFinal = countDataFinal.drop(columns=['siteName'])
countDataFinal['si_station_num'] = countDataFinal['si_station_num'].fillna("none")
countDataFinal['si_station_num'] = countDataFinal.apply(lambda row: f"ERN_{row.name}" if row['si_station_num'] == "none" else row['si_station_num'], axis=1)
countDataFinal = countDataFinal[(countDataFinal['si_station_num'] != "160605") & (countDataFinal['type'] != "dummy")]

# Visualize the data
fig, ax = plt.subplots()
countDataFinal.plot(ax=ax, column='type', legend=True)
plt.show()

# Incorporation of FTG Estimates to Create New Points -------------------------------

# Visualize the data
fig, ax = plt.subplots()
countDataFinal.plot(ax=ax, color='black')
SJTPO_blocksFTG.plot(ax=ax, column='FTG', alpha=0.5)
plt.show()

# Process assumes that FTG block centroids can be treated as if they are actual 
# point count locations. FTG estimates don't include combo ADT estimate, so that 
# is estimated based on average percentages of surrounding count locations.

countDataFinal_FTG = SJTPO_blocksFTG.copy()
countDataFinal_FTG['si_station_num'] = countDataFinal_FTG.index.map(lambda x: f"FTG_{x}")
countDataFinal_FTG['adt'] = 0
countDataFinal_FTG['truckADT'] = countDataFinal_FTG['FTG']
countDataFinal_FTG['comboADT'] = 0

# Visualize the data
fig, ax = plt.subplots()
countDataFinal_FTG.iloc[0].buffer(5280*2).boundary.plot(ax=ax)
countDataFinal.plot(ax=ax, color='red')
plt.show()

comboForJoin = pd.DataFrame()
for i in range(len(countDataFinal_FTG)):
    new = countDataFinal_FTG.iloc[i].buffer(5280*2).intersection(countDataFinal).summarize()
    new['comboADT'] = new['comboADT.1'].sum() / new['truckADT.1'].sum() * countDataFinal_FTG.iloc[i]['truckADT']
    comboForJoin = pd.concat([comboForJoin, new.round()])

countDataFinal_FTG['comboADT'] = comboForJoin['comboADT']

countDataFinal = pd.concat([countDataFinal, countDataFinal_FTG])

# Visualize the data
fig, ax = plt.subplots()
countDataFinal.plot(ax=ax, column='type', legend=True)
plt.show()

####  Start Here If Data Prepped ----------------------------------------------

In [1]:
# Save and load data
countDataFinal.to_pickle("countDataFinal.pkl")
countDataFinal = pd.read_pickle("countDataFinal.pkl")

NameError: name 'countDataFinal' is not defined

#### Actual Routing ----------------------------------------------------------

In [ ]:
target = "674-1"
X = 20

In [ ]:
# Define a function to geocode addresses
geolocator = Nominatim(user_agent="myGeocoder")

def geocode_address(address):
    location = geolocator.geocode(address)
    if location is not None:
        return location.latitude, location.longitude
    else:
        return None

In [ ]:
# Define a function to call a routing API
def call_here_routing_api(origin, destination):
    # Replace with your actual HERE API credentials
    app_id = "APP_ID"
    app_code = "APP_CODE"

    url = f"https://route.api.here.com/routing/7.2/calculateroute.json?waypoint0={origin}&waypoint1={destination}&mode=fastest;car;traffic:enabled&app_id={app_id}&app_code={app_code}"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return None

In [ ]:
# Define a function to calculate distance between two points
def haversine_distance(lat1, lon1, lat2, lon2):
    r = 6371  # Radius of the Earth in km
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
    return r * c

In [ ]:
# Geocode the target address
target_location = geocode_address(target)

# Filter the countDataFinal based on the target
target_data = countDataFinal[countDataFinal['si_station_num'] == target]

In [ ]:
# Calculate the distance from the target to all data points
distance_list = []
for index, row in countDataFinal.iterrows():
    lat1, lon1 = target_location
    lat2, lon2 = geocode_address(row['si_station_num'])
    distance = haversine_distance(lat1, lon1, lat2, lon2)
    distance_list.append(distance)

countDataFinal['distance_to_target'] = distance_list

In [ ]:
# Find the nearest X features along the distance of the target locations 
nearest_X = countDataFinal.sort_values(by='distance_to_target').iloc[1:X+1]

In [ ]:
# Call HERE routing API for each pair
route_data = []

for index, row in nearest_X.iterrows():
    origin = f"{target_location[0]},{target_location[1]}"
    destination = f"{row['latitude']},{row['longitude']}"
    route = call_here_routing_api(origin, destination)
    
    if route:
        route_data.append(route)

### Process the route data as needed -----------------------------------------

In [ ]:
# Map check to confirm results
# Create a map and add the route data to it using folium

m = folium.Map(location=target_location, zoom_start=10)

# Add the target location
folium.Marker(location=target_location, icon=folium.Icon(color='blue')).add_to(m)

# Add the route data
for route in route_data:
    # Extract the route coordinates from the response and add them as a Polyline
    route_coords = [(step['position']['latitude'], step['position']['longitude']) for step in route['response']['route'][0]['leg'][0]['maneuver']]
    folium.PolyLine(route_coords, color='red').add_to(m)

# Save the map to an HTML file
m.save('route_map.html')

# Apply routing function to all Classification count locations
# You can loop through all the stations and apply the routing function

results = []

for station in countDataFinal['si_station_num']:
    if station != target:
        route_result = RouteNearest(station, X)
        results.append(route_result)

# Union and Process Final Network -----------------------------------------